In [1]:
import numpy as np
import math
import cv2
import os
import sklearn
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn import svm

In [2]:

def compute_skew(src):
    """
    code source : https://gist.github.com/lnlonSA/d38f436552102a3fdf2a306f84ef1299
    """
    #load in grayscale:
#     src = cv2.imread(file_name,0)
    height, width = src.shape[0:2]
    
    #invert the colors of our image:
    cv2.bitwise_not(src, src)
    
    #Hough transform:
    minLineLength = width/2.0
    maxLineGap = 20
    lines = cv2.HoughLinesP(src,1,np.pi/180,100,minLineLength,maxLineGap)
    
    #calculate the angle between each line and the horizontal line:
    angle = 0.0
    nb_lines = len(lines)
    
    
    for line in lines:
        angle += math.atan2(line[0][3]*1.0 - line[0][1]*1.0,line[0][2]*1.0 - line[0][0]*1.0);
    
    angle /= nb_lines*1.0
    
    return angle* 180.0 / np.pi


In [3]:
skew_detected_female = []
for dirname, _, filenames in os.walk('/kaggle/input//cmp23-handwritten-males-vs-females/Female/Female/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename),0)
        if img is not None:
            skew_detected_female.append(compute_skew(img))
skew_detected_female = np.array(skew_detected_female)

skew_detected_male = []
for dirname, _, filenames in os.walk('/kaggle/input//cmp23-handwritten-males-vs-females/Male/Male/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename),0)
        if img is not None:
            skew_detected_male.append(compute_skew(img))
skew_detected_male = np.array(skew_detected_male)


In [4]:
# classifier
X=np.concatenate((skew_detected_male,skew_detected_female),axis=0)

Y=np.concatenate((np.ones(skew_detected_male.shape[0]),
                np.zeros(skew_detected_female.shape[0])),
                axis=0)

In [5]:
print(skew_detected_female.shape)

In [15]:
X,Y=sklearn.utils.shuffle(X,Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test

In [16]:
# print(X_train)
X_train = np.reshape(X_train,(X_train.shape[0], 1))
# print(X_train)

In [17]:
clf = svm.SVC(kernel='rbf', verbose=True, C=10)

clf.fit(X_train, y_train)

In [19]:
X_test = np.reshape(X_test,(X_test.shape[0], 1))

In [25]:
y_pred = clf.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))